# 🏋️ AI Search + Agent Service: Fitness-Fun Example 🤸

Welcome to our **AI Search + AI Agent** tutorial, where we'll:

1. **Create** an Azure AI Search index with fitness-oriented sample data
2. **Demonstrate** search functionality with fitness equipment queries
3. **Create** an AI agent with fitness knowledge and health disclaimers
4. **Show** how to have conversations about fitness advice and equipment recommendations

## 🏥 Health & Fitness Disclaimer
> **This notebook is for general demonstration and entertainment purposes, NOT a substitute for professional medical advice.**
> Always seek the advice of certified health professionals.

## Prerequisites
1. Complete Agent basics notebook - [1-basics.ipynb](1-basics.ipynb)
2. An **Azure AI Search** resource (formerly "Cognitive Search"), provisioned in your Azure AI Foundry project.

## What You'll Learn
- ✅ **Azure AI Search**: Create indexes, upload documents, perform searches
- ✅ **Agent Service**: Create AI agents with domain-specific knowledge
- ✅ **Integration Patterns**: How to connect search functionality with AI agents
- ✅ **Best Practices**: Error handling, resource cleanup, health disclaimers

## High-Level Flow
We'll do the following:
1. **Create** an AI Search index programmatically with sample fitness data.
2. **Upload** documents (fitness items) to the index.
3. **Verify** search functionality with test queries.
4. **Create** an AI agent with fitness expertise and proper disclaimers.
5. **Test** agent conversations about fitness equipment and advice.
6. **Clean up** resources responsibly.
 
 <img src="./seq-diagrams/5-ai-search.png" width="30%"/>

## 🔐 Authentication Setup

Before running the next cell, make sure you're authenticated with Azure CLI. Run this command in your terminal:

```bash
az login --use-device-code
```

This will provide you with a device code and URL to authenticate in your browser, which is useful for:
- Remote development environments
- Systems without a default browser
- Corporate environments with strict security policies

After successful authentication, you can proceed with the notebook cells below.

## 📋 Prerequisites: Create Azure AI Search Connection

Before proceeding with this tutorial, you need to create an Azure AI Search connection in your AI Foundry project.

### **Steps to Create the Connection:**

1. **Navigate to Azure AI Foundry portal**: https://ai.azure.com/
2. **Go to your project** → **Settings** → **Connections**
3. **Click "New Connection"** → **Azure AI Search**
4. **Provide the following details:**
   - **Connection Name**: Give it a descriptive name (e.g., "my-ai-search")
   - **Subscription**: Select your Azure subscription
   - **Resource Group**: Choose your resource group
   - **Azure AI Search Resource**: Select your existing search service or create a new one
5. **Click "Add Connection"** to complete the setup

### **Alternative: Use Azure CLI**
```bash
# Create AI Search connection via CLI
az ml connection create --file connection.yaml --resource-group <your-rg> --workspace-name <your-project>
```

### **⚠️ Important Notes:**
- The connection must be created **before** running the code below
- The code will automatically retrieve the default AI Search connection from your project
- If you don't have an Azure AI Search resource, you'll need to create one first

---


## 1. Create & Populate Azure AI Search Index
We'll create a minimal index called `myfitnessindex`, containing a few example items.
Make sure to set your environment variables for `SEARCH_ENDPOINT` and `SEARCH_API_KEY`. We'll use the `azure.search.documents.indexes` classes to manage the index schema. We'll also upload some sample data.


In [ ]:
# Import required Azure libraries
import os
from pathlib import Path
from dotenv import load_dotenv
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import SearchIndex, SimpleField, SearchFieldDataType, SearchableField
from azure.search.documents import SearchClient
from azure.identity import InteractiveBrowserCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import ConnectionType
from azure.ai.agents.models import AzureAISearchTool, AzureAISearchQueryType, MessageRole, ListSortOrder

# Load environment variables
load_dotenv(Path().parent.parent / '.env')

# Initialize project client with tenant-specific authentication
tenant_id = os.environ["TENANT_ID"]
credential = InteractiveBrowserCredential(tenant_id=tenant_id)

project_client = AIProjectClient(
    endpoint=os.environ["AI_FOUNDRY_PROJECT_ENDPOINT"],
    credential=credential
)

# Get search connection and create clients
search_conn = project_client.connections.get_default(
    connection_type=ConnectionType.AZURE_AI_SEARCH, 
    include_credentials=True
)

credential = AzureKeyCredential(search_conn.credentials['key'])
index_name = "myfitnessindex"

index_client = SearchIndexClient(
    endpoint=search_conn.target, 
    credential=credential
)

search_client = SearchClient(
    endpoint=search_conn.target,
    index_name=index_name,
    credential=credential
)

print(f"✅ Initialized clients for index: {index_name}")

✅ Initialized clients for index: myfitnessindex


## Create Search Index Schema

In [2]:
# Define index schema with fitness equipment fields
fields = [
    SimpleField(name="FitnessItemID", type=SearchFieldDataType.String, key=True),
    SearchableField(name="Name", type=SearchFieldDataType.String, filterable=True),
    SearchableField(name="Category", type=SearchFieldDataType.String, filterable=True, facetable=True),
    SimpleField(name="Price", type=SearchFieldDataType.Double, filterable=True, sortable=True, facetable=True),
    SearchableField(name="Description", type=SearchFieldDataType.String)
]

# Create index (delete if exists)
if index_name in [x.name for x in index_client.list_indexes()]:
    index_client.delete_index(index_name)
    print(f"🗑️ Deleted existing index: {index_name}")

index = SearchIndex(name=index_name, fields=fields)
created_index = index_client.create_index(index)
print(f"🎉 Created index: {created_index.name}")

🎉 Created index: myfitnessindex


## Upload Sample Documents

In [3]:
# Sample fitness equipment documents
sample_docs = [
    {
        "FitnessItemID": "1",
        "Name": "Adjustable Dumbbell",
        "Category": "Strength", 
        "Price": 59.99,
        "Description": "A compact, adjustable weight for targeted muscle workouts."
    },
    {
        "FitnessItemID": "2",
        "Name": "Yoga Mat",
        "Category": "Flexibility",
        "Price": 25.0,
        "Description": "Non-slip mat designed for yoga, Pilates, and other exercises."
    },
    {
        "FitnessItemID": "3",
        "Name": "Treadmill",
        "Category": "Cardio",
        "Price": 499.0,
        "Description": "A sturdy treadmill with adjustable speed and incline settings."
    },
    {
        "FitnessItemID": "4",
        "Name": "Resistance Bands",
        "Category": "Strength",
        "Price": 15.0,
        "Description": "Set of colorful bands for light to moderate resistance workouts."
    }
]

# Upload documents to index
result = search_client.upload_documents(documents=sample_docs)
print(f"🚀 Uploaded {len(sample_docs)} documents to index")


🚀 Uploaded 4 documents to index


## Verify Search Functionality

In [4]:
# Test search with "Strength" query
results = search_client.search(search_text="Strength", top=10)

print("🔍 Search results for 'Strength':")
print("-" * 40)
for doc in results:
    print(f"Name: {doc['Name']}")
    print(f"Category: {doc['Category']}")
    print(f"Price: ${doc['Price']:.2f}")
    print(f"Description: {doc['Description']}")
    print("-" * 40)

🔍 Search results for 'Strength':
----------------------------------------
Name: Resistance Bands
Category: Strength
Price: $15.00
Description: Set of colorful bands for light to moderate resistance workouts.
----------------------------------------
Name: Adjustable Dumbbell
Category: Strength
Price: $59.99
Description: A compact, adjustable weight for targeted muscle workouts.
----------------------------------------


## 2. Create Agent With Azure AI Search Tool

Configure the official AzureAISearchTool and create an AI agent with direct search integration.

In [5]:
# Configure Azure AI Search tool
ai_search_tool = AzureAISearchTool(
    index_connection_id=search_conn.id,
    index_name=index_name,
    query_type=AzureAISearchQueryType.SIMPLE,
    top_k=5,
    filter="",
)

# Create fitness agent with search capabilities
agent = project_client.agents.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    name="fitness-search-agent",
    instructions="""You are a Fitness Shopping Assistant with access to a live product catalog through Azure AI Search.

**Your Capabilities:**
- Search the fitness equipment catalog directly when users ask about products
- Provide fitness advice with proper health disclaimers
- Recommend equipment based on goals, budget, and safety

**Guidelines:**
- Always search for products when users ask about specific categories or items
- Include safety considerations and proper usage tips
- Remind users this is educational only - consult healthcare providers for medical advice

Be helpful, encouraging, and prioritize user safety!""",
    tools=ai_search_tool.definitions,
    tool_resources=ai_search_tool.resources,
)

print(f"🎉 Created agent: {agent.name} (ID: {agent.id})")
print(f"✅ Connected to search index: {index_name}")

🎉 Created agent: fitness-search-agent (ID: asst_oGXxnBGRKwJ7p3TwILypuDUg)
✅ Connected to search index: myfitnessindex


## 3. Test Agent with Search Integration

Test the agent with fitness equipment questions. The agent will automatically use the Azure AI Search tool when needed.

In [6]:
# Test the agent with fitness equipment questions
def test_agent_question(question):
    print(f"\n🔍 **Question:** {question}")
    print("-" * 50)
    
    # Create thread and send message
    thread = project_client.agents.threads.create()
    message = project_client.agents.messages.create(
        thread_id=thread.id,
        role=MessageRole.USER,
        content=question
    )
    
    # Process with agent
    run = project_client.agents.runs.create_and_process(
        thread_id=thread.id, 
        agent_id=agent.id
    )
    
    # Get response
    if run.status == "completed":
        messages = project_client.agents.messages.list(
            thread_id=thread.id, 
            order=ListSortOrder.DESCENDING
        )
        
        for msg in messages:
            if msg.role == "assistant":
                print("🤖 **Agent Response:**")
                for content in msg.content:
                    print(f"   {content.text.value}")
                break
    else:
        print(f"❌ Run failed with status: {run.status}")

# Test with sample questions
test_questions = [
    "What strength training equipment do you have available under $100?",
    "I'm looking for cardio equipment. What do you recommend?",
    "What's the cheapest fitness equipment you have?"
]

for question in test_questions:
    test_agent_question(question)

print("\n✅ Agent testing completed!")


🔍 **Question:** What strength training equipment do you have available under $100?
--------------------------------------------------
🤖 **Agent Response:**
   Here are some options for strength training equipment under $100:

1. **Resistance Bands**:
   - A set of colorful bands suitable for light to moderate resistance workouts. These are versatile and excellent for a full-body workout【3:0†source】.

2. **Adjustable Dumbbell**:
   - A compact option with adjustable weights, ideal for targeted muscle training while saving space【3:1†source】.

Both are budget-friendly options great for home fitness routines. Always follow safety guidelines during use to avoid injury, and consult a fitness expert if you are new to strength training!

🔍 **Question:** I'm looking for cardio equipment. What do you recommend?
--------------------------------------------------
🤖 **Agent Response:**
   Here are some options for strength training equipment under $100:

1. **Resistance Bands**:
   - A set of colo

In [ ]:
# Clean up resources
project_client.agents.delete_agent(agent.id)
index_client.delete_index(index_name)

print("🗑️ Deleted agent and search index")
print("✅ Cleanup completed!")

# 🎉 Congratulations!

You've successfully completed the **AI Search + Agent Service** tutorial using the **official Microsoft approach**!

## ✅ **What We Accomplished**

1. **🔍 Azure AI Search Integration**
   - Created a search index with fitness equipment data
   - Used the official `AzureAISearchTool` from `azure.ai.agents.models`
   - Configured with proper connection and query settings

2. **🤖 AI Agent with Search Capabilities**  
   - Created an agent using the official Microsoft approach
   - Agent automatically searches when users ask about products
   - Seamless integration with real-time search results

3. **📊 Key Features**
   - **Official SDK Support** - Uses documented, supported APIs
   - **Automatic Tool Calling** - Agent searches automatically when needed  
   - **Production Ready** - Clean, minimal code following best practices


Ready to build production AI applications with official Azure AI Search + Agent integration! 🚀